In [1]:
# Yuxin Lu, yuxinlu
import numpy as np
import pandas as pd
import os
from feature_extraction import read_emg_data, preprocess_emg, extract_features

In [2]:
print(os.listdir('./dataset'))
print([filename for filename in os.listdir('./dataset') if '_r_' in filename])
print(os.listdir(os.path.join('dataset', 's10_r_1')))

['s10_r_1', 's10_r_2', 's10_r_3', 's11_r_1', 's11_r_2', 's11_r_3', 's11_r_4', 's11_r_5', 's11_r_6', 's12_r_1', 's12_r_2', 's12_r_3', 's13_r_1', 's13_r_2', 's13_r_3', 's13_r_4', 's13_r_5', 's13_r_6', 's1_r_1', 's1_r_2', 's1_r_3', 's1_r_4', 's1_r_5', 's1_r_6', 's2_l_1', 's2_l_2', 's2_l_3', 's2_l_4', 's2_l_5', 's2_l_6', 's3_r_1', 's3_r_2', 's3_r_3', 's3_r_4', 's3_r_5', 's3_r_6', 's4_r_1', 's4_r_2', 's4_r_3', 's4_r_4', 's4_r_5', 's4_r_6', 's5_r_1', 's5_r_2', 's5_r_3', 's6_r_1', 's6_r_2', 's6_r_3', 's6_r_4', 's6_r_5', 's6_r_6', 's7_r_1', 's7_r_2', 's7_r_3', 's8_r_1', 's8_r_2', 's8_r_3', 's8_r_4', 's8_r_5', 's8_r_6', 's9_r_1', 's9_r_2', 's9_r_3']
['s10_r_1', 's10_r_2', 's10_r_3', 's11_r_1', 's11_r_2', 's11_r_3', 's11_r_4', 's11_r_5', 's11_r_6', 's12_r_1', 's12_r_2', 's12_r_3', 's13_r_1', 's13_r_2', 's13_r_3', 's13_r_4', 's13_r_5', 's13_r_6', 's1_r_1', 's1_r_2', 's1_r_3', 's1_r_4', 's1_r_5', 's1_r_6', 's3_r_1', 's3_r_2', 's3_r_3', 's3_r_4', 's3_r_5', 's3_r_6', 's4_r_1', 's4_r_2', 's4_r_3', 's

In [3]:
start_root = 'dataset'
# only consider right hands
folders_names = [foldername for foldername in os.listdir(start_root) if '_r_' in foldername]
X = list()
y = list()

# all possible gestures
codomains = ['paper', 'rock', 'scissors']

for foldername in folders_names:
    folder_path = os.path.join(start_root, foldername)
    for filename in os.listdir(folder_path):
        # only care about emg files
        if 'emg' not in filename:
            continue
        # extract the gt label from the filename
        y += [labelIdx for labelIdx, labelname in enumerate(codomains) if labelname in filename]
        # extract features
        file_path = os.path.join(folder_path, filename)
        raw_data = read_emg_data(file_path)
        filtered_data = preprocess_emg(raw_data.values)
        features = extract_features(filtered_data.T)
        X.append(features)

X = np.array(X)
print(X.shape)
print(X.reshape(X.shape[0], -1).shape)
print(X[0])
y = np.array(y)
print(y[0])


(1710, 7, 8)
(1710, 56)
[[2.27591847e+03 7.68869529e+03 1.34665242e+04 2.74055487e+03
  1.48398911e+03 9.62779070e+03 9.21911786e+03 3.04446265e+03]
 [6.33960576e+00 2.14169785e+01 3.75112095e+01 7.63385759e+00
  4.13367440e+00 2.68183585e+01 2.56799940e+01 8.48039734e+00]
 [2.80682070e+04 3.22770122e+05 8.72890139e+05 4.25620788e+04
  1.09380760e+04 5.61188659e+05 5.34648097e+05 5.98904299e+04]
 [1.20010227e+02 1.02704651e+02 1.27374577e+02 1.11259459e+02
  1.22835981e+02 1.13999332e+02 1.11006089e+02 1.15239789e+02]
 [2.84042793e+02 2.87589038e+02 2.62050384e+02 2.72500209e+02
  2.62864004e+02 3.11569942e+02 3.09335004e+02 2.53995371e+02]
 [7.81818938e+01 8.99080903e+02 2.43139859e+03 1.18554712e+02
  3.04679696e+01 1.56319961e+03 1.48927032e+03 1.66825632e+02]
 [8.84219546e+00 2.99846814e+01 4.93097237e+01 1.08884031e+01
  5.51979874e+00 3.95373192e+01 3.85910672e+01 1.29161027e+01]]
0


In [4]:
X = X.reshape(X.shape[0], -1)
print(X[0])
print(y[29])

[2.27591847e+03 7.68869529e+03 1.34665242e+04 2.74055487e+03
 1.48398911e+03 9.62779070e+03 9.21911786e+03 3.04446265e+03
 6.33960576e+00 2.14169785e+01 3.75112095e+01 7.63385759e+00
 4.13367440e+00 2.68183585e+01 2.56799940e+01 8.48039734e+00
 2.80682070e+04 3.22770122e+05 8.72890139e+05 4.25620788e+04
 1.09380760e+04 5.61188659e+05 5.34648097e+05 5.98904299e+04
 1.20010227e+02 1.02704651e+02 1.27374577e+02 1.11259459e+02
 1.22835981e+02 1.13999332e+02 1.11006089e+02 1.15239789e+02
 2.84042793e+02 2.87589038e+02 2.62050384e+02 2.72500209e+02
 2.62864004e+02 3.11569942e+02 3.09335004e+02 2.53995371e+02
 7.81818938e+01 8.99080903e+02 2.43139859e+03 1.18554712e+02
 3.04679696e+01 1.56319961e+03 1.48927032e+03 1.66825632e+02
 8.84219546e+00 2.99846814e+01 4.93097237e+01 1.08884031e+01
 5.51979874e+00 3.95373192e+01 3.85910672e+01 1.29161027e+01]
2


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb


np.random.seed(2024)
idices = list(range(y.shape[0]))
np.random.shuffle(idices)

train_idices = idices[:int(len(idices)*0.2)]
remain_idices = idices[int(len(idices)*0.2):]


clf = xgb.XGBClassifier(random_state=2024)
clf.fit(X[train_idices], y[train_idices])
y_pred = clf.predict(X[remain_idices])
print(accuracy_score(y[remain_idices], y_pred))
# SVC: 0.6703
# RandomForest: 0.8224
# GaussianNB: 0.6170
# XGBoost: 0.8194



0.8194444444444444
